In [27]:
import numpy as np
import sympy as sp
from scipy import optimize as opt
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import animation

from sympy.abc import x
from sympy.utilities.lambdify import lambdify, implemented_function
from sympy import Function
from sympy import Matrix

In [50]:
edge = np.genfromtxt('csv/adjacency.csv', delimiter=",").astype(np.int64) 
P = np.genfromtxt('csv/mdSpace.csv', delimiter=",") 
n = len(P)
L =np.genfromtxt('csv/eigVals.csv', delimiter=",") 
L_pos = np.array([L[i] if L[i]>0 else 0 for i in range(n)])
d = np.count_nonzero(L_pos)	# d ... the number of positive values
Ln = np.sqrt(L_pos)

f2 = np.array(Ln[0:d])
f2[::2] = 0
f1 = Ln[0:d] - f2
e1 = f1 / np.linalg.norm(f1)
e2 = f2 / np.linalg.norm(f2)
temp1 = e1
temp2 = e2

Xs = np.array([])
Ys = np.array([])

for i in np.arange(n):
    p0 = P[i,0:d]
    Xs = np.append(Xs,np.dot(p0,e1))
    Ys = np.append(Ys,np.dot(p0,e2))

In [52]:
sp.simplify(func)

(1.0*s**2 + 1.0*t**2 - 1)**2 + (0.488340027801521*a1 - 0.28108480633993*b1 - 0.361735245326884*c1 + 1)**2 + (0.488340027801521*a2 - 0.28108480633993*b2 - 0.361735245326884*c2 + 1)**2 + (1.0*a1*s + 1.0*b1*t - 0.488340027801521*c1*s + 0.28108480633993*c1*t - 1.0*s)**2 + (1.0*a2*s + 1.0*b2*t - 0.488340027801521*c2*s + 0.28108480633993*c2*t - 1.0*t)**2 + (1.0*a1*a2 - 0.488340027801521*a1*c2 - 0.488340027801521*a2*c1 + 1.0*b1*b2 + 0.28108480633993*b1*c2 + 0.28108480633993*b2*c1 + 0.361735245326884*c1*c2)**2 + ((0.15994778622916*a1 - 2.01168096268858e-16*c1)**2 + (0.15994778622916*a1 - 3.28738108536915e-16*c1)**2 + (0.15994778622916*a1 - 3.09725270170042e-17*c1)**2 + (0.15994778622916*a1 + 3.92523114670944e-17*c1)**2 + (0.15994778622916*a1 + 9.81307786677359e-17*c1)**2 + (0.933854125868679*a1 - 0.522929667786457*c1)**2 + (4.82898639082034e-9*b1 - 5.44113474592766e-9*c1)**2 + (0.162033895479706*b1 - 1.32476551201443e-16*c1)**2 + (0.162033895479707*b1 - 1.18370251767956e-16*c1)**2 + (0.1620338

In [53]:
identifier = ""

class DraggableCircle:
    def __init__(self, circle):
        self.circle = circle
        self.press = None

    def connect(self):
        'connect to all the events we need'
        self.cidpress = self.circle.figure.canvas.mpl_connect(
            'button_press_event', self.on_press)
        self.cidrelease = self.circle.figure.canvas.mpl_connect(
            'button_release_event', self.on_release)
        self.cidmotion = self.circle.figure.canvas.mpl_connect(
            'motion_notify_event', self.on_motion)

    def on_press(self, event):
        'on button press we will see if the mouse is over us and store some data'
        if event.inaxes != self.circle.axes: return

        contains, attrd = self.circle.contains(event)
        if not contains: return
        print('event contains', self.circle.center)
        x0, y0 = self.circle.center
        self.press = x0, y0, event.xdata, event.ydata
        global identifier
        identifier = self.circle.get_label()

    def on_motion(self, event):
        'on motion we will move the rect if the mouse is over us'
        if self.press is None: return
        if event.inaxes != self.circle.axes: return
        if(self.circle.get_label() == identifier):
            """
            # dragging single node and conectted links
            x0, y0, xpress, ypress = self.press
            dx = event.xdata - xpress
            dy = event.ydata - ypress
            #print('x0=%f, xpress=%f, event.xdata=%f, dx=%f, x0+dx=%f' %
            #      (x0, xpress, event.xdata, dx, x0+dx))
            self.circle.center = (x0+dx,y0+dy)
            self.circle.figure.canvas.draw()
            for i in range(len(edge)):
                if(edge[i,0]-1 == int(identifier)): 
                    edges[i].set_xdata((x0+dx,edges[i].get_xdata()[1]))
                    edges[i].set_ydata((y0+dy,edges[i].get_ydata()[1]))
                if(edge[i,1]-1 == int(identifier)): 
                    edges[i].set_xdata((edges[i].get_xdata()[0],x0+dx))
                    edges[i].set_ydata((edges[i].get_ydata()[0],y0+dy))
            
            """
            x0, y0, xpress, ypress = self.press
            x2 = x0 + event.xdata - xpress
            y2 = y0 + event.ydata - ypress
            p = P[int(identifier)]
            res = opt.minimize(g,arr,method='L-BFGS-B')
            e1 = res.x[0]*temp1 + res.x[1]*temp2 + res.x[2]*p
            e2 = res.x[3]*temp1 + res.x[4]*temp2 + res.x[5]*p
            temp1 = e1
            temp2 = e2
            for i in np.arange(n):
                p0 = P[i]
                global Xs, Ys
                Xs[i] = np.dot(p0,e1)
                Ys[i] = np.dot(p0,e2)
                nodes[i].center = (np.dot(p0,e1),np.dot(p0,e2))
                nodes[i].figure.canvas.draw()
                        
    def on_release(self, event):
        'on release we reset the press data'
        self.press = None
        self.circle.figure.canvas.draw()

    def disconnect(self):
        'disconnect all the stored connection ids'
        self.circle.figure.canvas.mpl_disconnect(self.cidpress)
        self.circle.figure.canvas.mpl_disconnect(self.cidrelease)
        self.circle.figure.canvas.mpl_disconnect(self.cidmotion)

In [54]:
gca = plt.gca()
nodes = np.array([])
edges = np.array([])

for i in range(n):
    circle = plt.Circle((Xs[i], Ys[i]), radius=0.2, fc='y',label=str(i))
    gca.add_patch(circle)
    nodes = np.append(nodes,circle)

for e in edge:
    line = plt.Line2D((Xs[e[0]-1], Xs[e[1]-1]), (Ys[e[0]-1],Ys[e[1]-1]), lw=1)
    gca.add_line(line)
    edges = np.append(edges,line)
    
dcs = []
for node in nodes:
    dc = DraggableCircle(node)
    dc.connect()
    dcs.append(dc)

plt.axis('scaled')
plt.show()

event contains (0.4844625954635699, 0.28788830799645709)
event contains (-1.3744218090843257, 0.76451543320541149)
event contains (-1.345128005477495, 0.70081375585117922)
event contains (-1.3008336436928962, 0.88981243678588995)
event contains (-1.365682707703179, 0.74503710005062396)
event contains (-1.3744218090843257, 0.76451543320541149)
event contains (-1.345128005477495, 0.70081375585117922)
event contains (-1.3008336436928962, 0.88981243678588995)
event contains (-1.365682707703179, 0.74503710005062396)


In [59]:
x2 = Xs[14] * 0.9
y2 = Ys[14] * 0.9
p = P[14]
res = opt.minimize(g,arr,method='L-BFGS-B')
print(res)
e1 = res.x[0]*temp1 + res.x[1]*temp2 + res.x[2]*p
e2 = res.x[3]*temp1 + res.x[4]*temp2 + res.x[5]*p
temp1 = e1
temp2 = e2
for i in np.arange(n):
    p0 = P[i]
    global Xs, Ys
    Xs[i] = np.dot(p0,e1)
    Ys[i] = np.dot(p0,e2)
    nodes[i].center = (np.dot(p0,e1),np.dot(p0,e2))

      fun: 0.59057987574475646
 hess_inv: <8x8 LbfgsInvHessProduct with dtype=float64>
      jac: array([ -7.56061880e-06,  -1.44328993e-06,   3.16413562e-06,
         2.66453526e-07,  -3.08642001e-06,  -1.55431223e-06,
         1.63202785e-06,  -1.77635684e-06])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 504
      nit: 48
   status: 0
  success: True
        x: array([-0.22620468, -0.64355453,  1.53016793,  0.22618662,  0.64356298,
        1.14074427,  0.99420173, -0.10753306])


In [60]:
lam_f(*res.x)

0.59057987574475646

In [39]:
Ys

array([-0.56897311, -1.56451723, -1.50544376, -1.50811793, -1.69731664,
        0.28788831,  0.7715399 ,  0.74345398,  0.79191257,  0.86831038,
        0.28108481,  0.70081376,  0.76451543,  0.88981244,  0.7450371 ])

In [41]:
Xs

array([-0.00268292,  0.10425172, -0.15400581,  0.00563615,  0.01452715,
       -0.20190668, -0.373379  , -0.40135545, -0.64987464, -0.8022875 ,
        0.2045896 ,  0.49591569,  0.39260067,  0.36797838,  0.99999265])

In [42]:
Ys

array([-0.57065517, -1.5647251 , -1.52086376, -1.51656633, -1.69179231,
        0.21617485,  0.58318783,  0.555932  ,  0.59035477,  0.65479044,
        0.35448032,  0.89498665,  0.94972351,  1.06497149,  1.00000082])

array([  0.00000000e+00,   9.46011143e-01,   3.85404712e-06,
         1.62035950e-01,   5.51658864e-06,   1.62036313e-01,
        -1.77354681e-06,   1.62030330e-01,  -3.81922132e-06,
         1.62029871e-01,  -1.57421221e-06,   4.82903088e-09])

In [ ]:
x0, y0, xpress, ypress = self.press
        global Xs, Ys, x2, y2, nodes, edges, dcs, gca, e1, e2, temp1, temp2
        x2 = event.xdata
        y2 = event.ydata
        p = P[int(identifier)-1]
        res = opt.minimize(g,arr,method='L-BFGS-B')
        e1 = res.x[0]*temp1 + res.x[1]*temp2 + res.x[2]*p
        e2 = res.x[3]*temp1 + res.x[4]*temp2 + res.x[5]*p
        temp1 = e1
        temp2 = e2
        Xs = np.array([])
        Ys = np.array([])
        for i in np.arange(n):
            p0 = P[i,0:d]
            Xs = np.append(Xs,np.dot(p0,e1))
            Ys = np.append(Ys,np.dot(p0,e2))
        nodes = np.array([])
        edges = np.array([])
        for i in np.arange(n):
            circle = plt.Circle((Xs[i], Ys[i]), radius=0.2, fc='y',label=str(i))
            gca.add_patch(circle)
            nodes = np.append(nodes,circle)
        for e in edge:
            line = plt.Line2D((Xs[e[0]-1], Xs[e[1]-1]), (Ys[e[0]-1],Ys[e[1]-1]), lw=1)
            gca.add_line(line)
            edges = np.append(edges,line)
        dcs = []
        for node in nodes:
            dc = DraggableCircle(node)
            dc.connect()
            dcs.append(dc)
        plt.axis('scaled')
        plt.show()

In [10]:
x,y,x2,y2 = (Xs[1],Ys[1],1.5,1.5)
p = P[1]
a1,b1,c1,a2,b2,c2,t,s = sp.symbols('a1 b1 c1 a2 b2 c2 t s')
var = [a1,b1,c1,a2,b2,c2,t,s]
E1 = a1*e1 + b1*e2 + c1*p
E2 = a2*e1 + b2*e2 + c2*p
r  = s*e1 + t*e2

In [11]:
f = Matrix([
		np.dot(E1,E1) - 1,
		np.dot(E2,E2) - 1,
		np.dot(E1,E2),
		np.dot(r,r) - 1,
		np.dot(E1,r) - np.dot(e1,r),
		np.dot(E2,r) - np.dot(e2,r),
		np.dot(p,E1) - x2,
		np.dot(p,E2) - y2
		])

In [12]:
df = np.array([])
for i in range(len(f)):
	for j in range(len(var)):
		df = np.append(df, sp.diff(f[i],var[j]))

In [13]:
df = df.reshape(len(var),len(f))

In [14]:
type(f)

sympy.matrices.dense.MutableDenseMatrix

In [15]:
func=f.norm()
lam_f = lambdify((a1,b1,c1,a2,b2,c2,t,s), func, 'numpy')

In [73]:
lam_f

<function numpy.<lambda>>

In [72]:
#bnds = ((-1, 1), (-1, 1), (-1, 1), (-1, 1),(-1, 1), (-1, 1), (-1, 1), (-1, 1))

print(lam_f(1,1,1,1,1,1,1,1))
arr = np.array([1,1,1,1,1,1,1,1])

def g(args): return lam_f(*args)
opt.minimize(g,arr,method='L-BFGS-B')

1.78292713414


      fun: 5.9874912320374349e-07
 hess_inv: <8x8 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.04601905, -0.00382544,  0.0051648 ,  0.00240156, -0.01345265,
        0.01312605,  0.01092906, -0.0009878 ])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 1206
      nit: 72
   status: 0
  success: True
        x: array([  1.00000005e+00,  -4.68082994e-06,  -7.93041648e-06,
         2.22750609e-07,   9.99991456e-01,  -1.48720468e-05,
         8.89506180e-01,   4.56923134e-01])

In [87]:
type(func)

sympy.core.power.Pow

In [88]:
lambdify((a1,b1,c1,a2,b2,c2,t,s),func)

<function numpy.<lambda>>

In [36]:
sp.var('x y')
f = sp.lambdify((x, y), x**2 + y**2)

In [38]:
f(2, 2)

8

In [34]:
def g(args): return lam_f(*args)

In [31]:
g([2, 2])

8

In [32]:
g(np.array([2, 2]))

8

In [40]:
def g(args): return lam_f(*args)
res = opt.minimize(g,np.array([1,1,1,1,1,1,1,1]),method='L-BFGS-B')

In [41]:
res

      fun: 5.7727004917249111e-07
 hess_inv: <8x8 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 0.0391516 , -0.00496174,  0.00608893, -0.0118562 ,  0.00776431,
        0.00232543,  0.01370445,  0.00019245])
  message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 1206
      nit: 72
   status: 0
  success: True
        x: array([  1.00000005e+00,  -4.52579073e-06,  -7.67812664e-06,
         2.02142522e-07,   9.99991771e-01,  -1.43286721e-05,
         8.89507923e-01,   4.56919742e-01])

In [44]:
res.x[0]

1.000000045503445

In [45]:
a1

a1

TypeError: 'numpy.ndarray' object is not callable